In [19]:
import igl
import numpy as np
import meshplot as mp

from collections import deque

In [20]:
# build graph of geodesic distances on a mesh
def build_geodesic_graph(V, F):
    n = V.shape[0]  # number of vertices
    adj = [[] for _ in range(n)]  # adjacency list: for each vertex, list of (neighbor, weight)

    # Compute triangle-triangle adjacency for dual edges
    TT, _ = igl.triangle_triangle_adjacency(F)

    # For each face
    for f_idx in range(F.shape[0]):
        tri = F[f_idx] 

        # For each corner of the face
        for loc in range(3):
            v  = tri[loc]            # current vertex index
            w1 = tri[(loc + 1) % 3]  # next vertex index around the triangle
            w2 = tri[(loc + 2) % 3]  # other vertex index

            # Primal edges
            for w in (w1, w2):
                d = np.linalg.norm(V[v] - V[w])
                # add primal edge
                adj[v].append((w, d))
                adj[w].append((v, d))

            # Dual edges
            f2 = int(TT[f_idx, loc])  # adjacent face across edge opposite v
            if f2 < 0:
                # boundary edge: no adjacent face
                continue

            # find the vertex in face f2 that is not w1 or w2
            edge_vs = {w1, w2}
            vp = next(int(x) for x in F[f2] if int(x) not in edge_vs)

            # Compute vectors for angle calculations at shared edge vertex w1
            w = w1
            vec_vw = V[v] - V[w]      # vector v->w
            vec_uw = V[w2] - V[w]     # vector w2->w (edge direction)
            vec_vpw = V[vp] - V[w]    # vector vp->w (across triangle f2)

            # norms of these vectors
            nv  = np.linalg.norm(vec_vw)
            nu  = np.linalg.norm(vec_uw)
            npv = np.linalg.norm(vec_vpw)

            # corner angles at w in both triangles
            alfa = np.arccos(np.clip(np.dot(vec_vw, vec_uw) / (nv * nu), -1, 1))
            beta = np.arccos(np.clip(np.dot(vec_vpw, vec_uw) / (npv * nu), -1, 1))

            # Geodesic length between v and vp via w
            dual_len = np.sqrt(nv**2 + npv**2 - 2 * nv * npv * np.cos(alfa + beta))
    
            # add dual edge
            adj[v].append((vp, dual_len))
            adj[vp].append((v, dual_len))
    return adj

# Compute geodesic distances from sources
def geodesic_distances(adj, sources):
    n = len(adj)
    dist = np.full(n, np.inf)  # initialize all distances to infinity
    Q = deque()                # deque for SLF-LLL

    # Initialize queue with all source vertices at distance 0
    for s in sources:
        dist[s] = 0.0
        Q.append(s)

    # Propagate distances
    while Q:
        u = Q.popleft()
        du = dist[u]
        # relax all outgoing edges from u
        for v, w in adj[u]:
            alt = du + w
            if alt < dist[v]:
                dist[v] = alt
                # SLF–LLL: push to front if smaller than current front
                if Q and alt < dist[Q[0]]:
                    Q.appendleft(v)
                else:
                    Q.append(v)

    return dist


In [21]:
# extract path from target to a source
def extract_geodesic_path(target, sources, dist, V, F):
    path = []


    
    return path
    

def plot_geodesic_path(V, F, dist, path, src, target):
    p = mp.plot(V, F, c=dist, shading={"wireframe": False})

    # mark sources
    p.add_points(V[src, :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.02})
    # mark target
    p.add_points(V[[target], :], c=np.array([[0.0, 1.0, 0.0]]), shading={"point_size": 0.02})

    # draw path
    if len(path) >= 2:
        for i in range(len(path) - 1):
            a = V[path[i]]
            b = V[path[i+1]]
            p.add_lines(a, b, shading={"line_width": 0.05})

    return p


In [23]:
# Load the test mesh
V, F = igl.read_triangle_mesh("./data/bunny_1k.obj")
#mp.plot(V, F, shading={"wireframe": True})

In [24]:
# compute geodesic distances
adj = build_geodesic_graph(V, F)

src = [0]
tgt = 9

dist = geodesic_distances(adj, src)

# apply color map
plotter = mp.plot(V, F, c=dist, shading={"wireframe": False})

# mark the source with a red point
plotter.add_points(V[[src], :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…

1

In [ ]:
src = [0]
tgt = 300

# Extraxt geodesic path from target to source
path = extract_geodesic_path(tgt, src, dist, V, F)

# Visualize the geodesic path
plot_geodesic_path(V, F, dist, path, src, tgt)

TypeError: 'numpy.intc' object is not iterable